In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from scipy import spatial
from scipy import sparse
from scipy import linalg, mat
from tqdm import tqdm, notebook
import math
import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from random import randint
from src.utils import split_in, extended_describe, cos_cdist
import lightfm as lfm
from lightfm import data
from lightfm import cross_validation
from lightfm import evaluation

In [4]:
df_applicants_train = pd.read_csv("../data/02_intermediate/postulaciones/postulaciones_extended_train.csv")
df_applicants_to_predict = pd.read_csv("../data/02_intermediate/postulaciones/postulaciones_test.csv")
df_notice = pd.read_csv("../data/02_intermediate/avisos/avisos_extended.csv")

In [5]:
columns_dict = pd.read_csv('../data/02_intermediate/postulantes/atributos_diccionario.csv')
matrix_train = pd.read_csv('../data/02_intermediate/postulantes/postulantes_matrix_train.csv', index_col='idpostulante')
matrix_test = pd.read_csv('../data/02_intermediate/postulantes/postulantes_matrix_test.csv', index_col='idpostulante')
numpy_matrix_test = matrix_test.to_numpy()
numpy_matrix_train = matrix_train.to_numpy()
normalized_test = normalize(numpy_matrix_test, norm='l2')
normalized_train = normalize(numpy_matrix_train, norm='l2')

In [56]:
index = 1
applicant_to_predict_vector = normalized_test[index] # matrix_test.iloc[index,].values
applicant_to_predict_id = matrix_test.iloc[index,].name
treshold = 0.9
top = 10
cos = cos_cdist(normalized_train, applicant_to_predict_vector)
sim = pd.DataFrame({'idpostulante': matrix_train.index.values, 'similaridad': cos})
top_applicants = sim[(sim.similaridad > treshold)].head(top).idpostulante.values
top_notices = df_applicants_train[df_applicants_train.idpostulante.isin(top_applicants)].idaviso.unique()
notices_temp = df_applicants_train[df_applicants_train.idpostulante.isin(top_applicants)].groupby('idaviso').agg({'idpostulante': 'count'}).reset_index()
notices_ids = notices_temp[notices_temp.idpostulante > 1].idaviso.values[:10]
notcies_len = len(notices_ids)
applicant = df_applicants_to_predict[df_applicants_to_predict.idpostulante==applicant_to_predict_id]

In [57]:
b = pd.DataFrame({'idaviso': notices_ids, 'idpostulante': [applicant.idpostulante.values[0]] * notcies_len})

In [58]:
a

,idaviso,idpostulante
0,1112047992,0z5JW1r
1,1112242227,0z5JW1r
2,1112252474,0z5JW1r
3,1112252928,0z5JW1r
4,1112263302,0z5JW1r
5,1112267901,0z5JW1r
6,1112268533,0z5JW1r
7,1112270461,0z5JW1r
8,1112274976,0z5JW1r
9,1112277509,0z5JW1r


In [59]:
b

,idaviso,idpostulante
0,1111772511,0zB01pE
1,1111985266,0zB01pE
2,1112249085,0zB01pE
3,1112263314,0zB01pE
4,1112267868,0zB01pE
5,1112269953,0zB01pE
6,1112278019,0zB01pE
7,1112281653,0zB01pE
8,1112282187,0zB01pE
9,1112288726,0zB01pE


In [60]:
pd.concat([a, b])

,idaviso,idpostulante
0,1112047992,0z5JW1r
1,1112242227,0z5JW1r
2,1112252474,0z5JW1r
3,1112252928,0z5JW1r
4,1112263302,0z5JW1r
5,1112267901,0z5JW1r
6,1112268533,0z5JW1r
7,1112270461,0z5JW1r
8,1112274976,0z5JW1r
9,1112277509,0z5JW1r


In [ ]:
ds = lfm.data.Dataset()
ds.fit(users=df_applicants_train.idpostulante.unique(), items=df_notice.idaviso.unique())
ds.interactions_shape()

In [ ]:
# transformar  'fechapostulacion' en un int
(interactions, weights) = ds.build_interactions(df_applicants_train[['idpostulante', 'idaviso']].itertuples(index=False))
interactions

In [ ]:
train, test = lfm.cross_validation.random_train_test_split(interactions, test_percentage=0.2)
model = lfm.LightFM(no_components=20)
model.fit(train, epochs=50, num_threads=8)

In [ ]:
train_precision = lfm.evaluation.precision_at_k(model, train, k=10, num_threads=8).mean()
test_precision  = lfm.evaluation.precision_at_k(model, test,  k=10, num_threads=8).mean()
print(f'Precision en train: {train_precision}')
print(f'Precision en test: {test_precision}')